<a href="https://colab.research.google.com/github/20-hayato/practice/blob/master/speech_recog_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install SpeechRecognition
!pip install pydub
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [2]:
from zmq.constants import RECONNECT_IVL
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from os import path
from pydub import AudioSegment
import subprocess
from datetime import datetime, timedelta, timezone

RECORD_DIR = "record_data/"
JST = timezone(timedelta(hours=+9), 'JST')

def record_js(filename='record'):
  date = datetime.now(JST).strftime('%Y%m%d_%H:%M:%S')
  js = Javascript('''
    async function record() {
      let rec;
      let chanks;


      // HTML組み立て
      const div = document.createElement('div');
      const startRecord = document.createElement('button');
      startRecord.textContent = 'Rec';
      div.appendChild(startRecord);

      const stopRecord = document.createElement('button');
      stopRecord.textContent = 'Stop';
      stopRecord.style.display = 'none'
      div.appendChild(stopRecord);

      const audio = document.createElement('audio');
      div.appendChild(audio);

      document.body.appendChild(div);

      // Audioが有効になったら
      function handlerFunction(stream,resolve) {
          rec = new MediaRecorder(stream);
          // 録音が完了したら
          rec.ondataavailable = e => {
              chanks.push(e.data);
              if (rec.state == "inactive") {
                  let blob = new Blob(chanks, { type: 'audio/mpeg-3' });
                  audio.src = URL.createObjectURL(blob);
                  audio.controls = true;
                  audio.autoplay = true;
                  resolve();
              }
          }
      }

      startRecord.onclick = e => {
          startRecord.style.display = 'none'
          stopRecord.style.display = 'block'
          chanks = [];
          rec.start();
      }

      stopRecord.onclick = e => {
        startRecord.style.display = 'block'
        stopRecord.style.display = 'none'
        rec.stop();
      }

      function blobToBase64(blob) {
        return new Promise((resolve, _) => {
          const reader = new FileReader();
          reader.onloadend = () => resolve(reader.result);
          reader.readAsDataURL(blob);
        });
      }

      await new Promise((resolve) => {
        navigator.mediaDevices.getUserMedia({ audio: true })
            .then(stream => { handlerFunction(stream,resolve) })
      });
      let blob = new Blob(chanks, { type: 'audio/mpeg-3' });
      return await blobToBase64(blob);
    }
    ''')
  display(js)
  data = eval_js('record()')
  binary = b64decode(data.split(',')[1])

  filename_date = RECORD_DIR + filename + "_" + date + '.mp3'
  with open(filename_date, 'wb') as f:
    f.write(binary)
  return filename_date

def mp3_to_wav(src):
  filename = src.split('.', 1)[0]
  dst = filename +'.wav'
  subprocess.call(["ffmpeg", "-i", src, dst])
  return dst

In [3]:
import speech_recognition as sr
import glob

r = sr.Recognizer()

try:
  filename_mp3 = record_js()
  filename_wav = mp3_to_wav(filename_mp3)

  print("waiting for converting...")
  while glob.glob(filename_wav, recursive=False) == []:
    pass

  audio_file = sr.AudioFile(filename_wav)
  with audio_file as source:
      r.adjust_for_ambient_noise(source) #雑音対策
      audio = r.record(source)
  print("result: ")
  print(r.recognize_google(audio, language='ja-JP'))
except Exception as err:
  print(str(err))


<IPython.core.display.Javascript object>

waiting for converting...
result: 
あいうえ たちつてとご飯は何ですか


In [32]:
# 消去用スクリプト
!ls ./record_data/
!rm ./record_data/*

record_20231113_16:01:17.mp3  record_20231113_16:01:17.wav
